In [7]:
import pandas as pd
from transformers import pipeline

2024-02-27 08:35:57.391836: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-27 08:35:57.570200: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 08:35:57.570330: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 08:35:57.602454: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-27 08:35:57.699580: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-27 08:35:57.703163: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [8]:
sgs = pd.read_csv('../working-csvs/fomc_sents.csv', index_col=0)

In [9]:
sgs

,lname,date,section,content
0,bernanke,2002-08-13,1,"Thank you, Mr. Chairman."
0,bernanke,2002-08-13,1,As we search for the signal of an incipient re...
0,bernanke,2002-08-13,1,The first source of noise is the financial mar...
0,bernanke,2002-08-13,1,Our financial markets usually exhibit controll...
0,bernanke,2002-08-13,1,The stock market in particular seems to be sea...
...,...,...,...,...
6701,yellen,2018-01-31,2,Further comments?
6701,yellen,2018-01-31,2,Okay.
6701,yellen,2018-01-31,2,"Well, hearing this discussion, I think my incl..."
6701,yellen,2018-01-31,2,I recognize that there is some risk in doing i...


In [10]:
pipe = pipeline("text-classification", model="ProsusAI/finbert")

In [11]:
sgs['sentiment'] = pipe(sgs['content'].tolist())

runtime: 189m 7.2s

In [19]:
sgs['sent_category'] = sgs['sentiment'].map(lambda x : x['label'])
sgs['sent_prob'] = sgs['sentiment'].map(lambda x : x['score'])

In [20]:
def sent_map(category):
    if category == 'neutral':
        return 0
    elif category == 'positive':
        return 1
    return -1

In [24]:
sgs['sent'] = sgs['sent_category'].map(sent_map)

In [34]:
sgs.drop(columns=['sentiment', 'sent_category'], inplace=True)

In [36]:
sgs.to_csv('../working-csvs/fomc-sents-w-sentiment.csv')